In [153]:
import os
import re
import urllib.request
from bs4 import BeautifulSoup
import os
import csv
import time
import requests
from urllib.request import Request, urlopen
import pandas as pd

In [167]:
topic_Name = ["Data-Science","machine-Learning","Open-CV","Computer-Vision","GAN","variational-encoder","awsome","javascript","c++"]
number_of_Repositories  = 2
repository_list = {}
for name in topic_Name:
    repository_list[name] = []
Data = {}
col_name = ["topic","name","user","star","fork","watch","topic_tag","discription_text","discription_url","commits","branches","packages","releases","contributers","License","url"]
for name in col_name:
    Data[name] = []
Data_f = pd.DataFrame(Data)

In [168]:
for topic in topic_Name:
    page_number = 1
    repo_number = 0
    temp_data = {}
    for name in col_name:
        temp_data[name] = []
    while len(repository_list[topic]) < number_of_Repositories:
        url = "https://github.com/search?o=desc&p="+ str(page_number) +"&q="+str(topic)+"&s=stars&type=Repositories"
        data_page = requests.get(url)
        time.sleep(0.1)
        main_data = BeautifulSoup(data_page.content,'html.parser')
        for sub_data in main_data.findAll('li',attrs={'class':'repo-list-item'}):
            try:
                repository_path = str(sub_data.findAll('a',attrs={'class':"v-align-middle"})[0].text)
                repository_user_name , repository_name = repository_path.split('/')
                repository_URL = "https://github.com/" + repository_path
                repository_page = requests.get(repository_URL)
                time.sleep(0.1)
                repository_data = BeautifulSoup(repository_page.content,'html.parser')
                # ropository data

                ## Social Count
                social_count = repository_data.findAll('a',attrs={'class':"social-count"})
                repository_watch = social_count[0].text.strip()
                repository_star = social_count[1].text.strip()
                repository_fork = social_count[2].text.strip()
                # print(repository_watch,repository_star,repository_fork)
                
                # Repository tag
                repository_topic_tag = [str(i.text)[3:-1] for i in repository_data.findAll('a',attrs={'class':"topic-tag"})]
                repository_discription = repository_data.find('div',attrs={'class':"f4"})

                # Repository Discription
                repository_discription_text = repository_discription.find('span',attrs={'class':"mr-2"}).text.strip() if repository_discription.find('span',attrs={'class':"mr-2"}) != None else None
                repository_discription_url = repository_discription.find('a').text if repository_discription.find('a') != None else None
                
                # Repository overall-summary
                repository_overall_summary = repository_data.find('ul',attrs={'class':'numbers-summary'})
                repository_num_summary = repository_overall_summary.findAll('span',attrs={'class':'num'})
                repository_commits = repository_num_summary[0].text.strip() if repository_num_summary[0] != None else 0
                repository_branches = repository_num_summary[1].text.strip() if repository_num_summary[1] != None else 0
                repository_packages = repository_num_summary[2].text.strip() if repository_num_summary[2] != None else 0
                repository_releases = repository_num_summary[2].text.strip() if repository_num_summary[3] != None else 0
                repository_contributers = repository_num_summary[3].text.strip() if repository_num_summary[3] != None else 0
                repository_License = repository_data.find('ul',attrs={'class':'numbers-summary'}).findAll('li')[-1].text.strip()

                # print(repo_number,repository_path,repository_URL,repository_name,repository_star,repository_fork,repository_watch,repository_topic_tag,repository_discription_text,repository_discription_url,repository_commits,repository_branches,repository_packages,repository_releases,repository_contributers,repository_License)
                
                for a,b in zip(["topic","name","user","star","fork","watch","topic_tag","discription_text","discription_url","commits","branches","packages","releases","contributers","License","url"],[topic,repository_name,repository_user_name,repository_star,repository_fork,repository_watch,repository_topic_tag,repository_discription_text,repository_discription_url,repository_commits,repository_branches,repository_packages,repository_releases,repository_contributers,repository_License,repository_URL]):
                    temp_data[a].append(b)
                    
                repository_list[topic].append(repository_path)
                repo_number += 1
            except:
                continue
            if (repo_number >= number_of_Repositories):
                break
        page_number += 1
    temp_df = pd.DataFrame(temp_data)
    Data_f = pd.concat([Data_f,temp_df])

In [156]:
Data_f = pd.Da# taFrame(Data)

In [169]:
Data_f

,topic,name,user,star,fork,watch,topic_tag,discription_text,discription_url,commits,branches,packages,releases,contributers,License,url
0,Data-Science,keras,keras-team,47.9k,18.1k,2.1k,"[deep-learning, tensorflow, neural-networks, m...",Deep Learning for humans,http://keras.io/,"5,342",6,0,0,49,View license,https://github.com/keras-team/keras
1,Data-Science,scikit-learn,scikit-learn,40.3k,19.6k,2.2k,"[machine-learning, python, statistics, data-sc...",scikit-learn: machine learning in Python,https://scikit-learn.org,"25,403",22,0,0,108,View license,https://github.com/scikit-learn/scikit-learn
0,machine-Learning,tensorflow,tensorflow,144k,80.9k,8.4k,"[tensorflow, machine-learning, python, deep-le...",An Open Source Machine Learning Framework for ...,https://tensorflow.org,"84,317",50,0,0,107,Apache-2.0,https://github.com/tensorflow/tensorflow
1,machine-Learning,keras,keras-team,47.9k,18.1k,2.1k,"[deep-learning, tensorflow, neural-networks, m...",Deep Learning for humans,http://keras.io/,"5,342",6,0,0,49,View license,https://github.com/keras-team/keras
0,Open-CV,SmartOpenCV,HuTianQi,1.8k,509,93,"[opencv, ai, android, cv, sdk, surfaceview, ca...",🔥 🔥 🔥 SmartOpenCV是一个OpenCV在Android端的增强库，解决了Ope...,None,20,1,0,0,2,Apache-2.0,https://github.com/HuTianQi/SmartOpenCV
1,Open-CV,OpenCV-Python-Tutorial,makelove,1.8k,733,89,"[python, opencv, tutorial, imread, imshow, vid...","OpenCV问答群,QQ群号:187436093",https://space.bilibili.com/180948619,257,2,0,0,0,MIT,https://github.com/makelove/OpenCV-Python-Tuto...
0,Computer-Vision,opencv,opencv,43.9k,34.9k,2.6k,"[opencv, c-plus-plus, computer-vision, deep-le...",Open Source Computer Vision Library,https://opencv.org,"28,487",3,0,0,93,View license,https://github.com/opencv/opencv
1,Computer-Vision,d2l-zh,d2l-ai,17k,4.4k,830,"[deep-learning, book, notebook, natural-langua...",《动手学深度学习》：面向中文读者、能运行、可讨论。英文版即伯克利“深度学习导论”教材。,http://zh.d2l.ai,"2,986",1,0,0,10,Apache-2.0,https://github.com/d2l-ai/d2l-zh
0,GAN,pytorch-CycleGAN-and-pix2pix,junyanz,11.6k,3.4k,311,"[pytorch, gan, cyclegan, pix2pix, deep-learnin...",Image-to-Image Translation in PyTorch,None,445,3,0,0,0,View license,https://github.com/junyanz/pytorch-CycleGAN-an...
1,GAN,CycleGAN,junyanz,9.6k,1.6k,401,"[gan, generative-adversarial-network, deep-lea...",Software that can generate photos from paintin...,None,95,1,0,0,0,View license,https://github.com/junyanz/CycleGAN
